In [41]:
import pandas as pd
import pickle

In [42]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/plbart_base_embeddings/df_dc_embeded_by_line_plbart.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_34947/2512578579.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [43]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,8077,critical,public class DruidPooledCallableStatement exte...,1,"[[710.2053833007812, -799.0264892578125, 321.9...","[[0.7653075456619263, -0.8610199093818665, 0.3..."
1,5553,critical,public interface IAopReferenceModel {\n\t\n\tv...,1,"[[32.499385833740234, -1.5158289670944214, -24...","[[1.8055214881896973, -0.08421272039413452, -1..."
2,9341,critical,@XmlAccessorType(XmlAccessType.FIELD)\n@XmlTyp...,1,"[[124.71330261230469, -38.58845138549805, -42....","[[3.3706297874450684, -1.042931079864502, -1.1..."
3,10419,critical,public class Ipam {\n private String driver;\...,1,"[[130.8542022705078, -74.68362426757812, 18.32...","[[1.9243265390396118, -1.0982885360717773, 0.2..."
4,12232,critical,public class DruidPooledCallableStatement exte...,1,"[[710.2053833007812, -799.0264892578125, 321.9...","[[0.7653075456619263, -0.8610199093818665, 0.3..."


In [44]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,8077,critical,1,"[[710.2053833007812, -799.0264892578125, 321.9..."
1,5553,critical,1,"[[32.499385833740234, -1.5158289670944214, -24..."
2,9341,critical,1,"[[124.71330261230469, -38.58845138549805, -42...."
3,10419,critical,1,"[[130.8542022705078, -74.68362426757812, 18.32..."
4,12232,critical,1,"[[710.2053833007812, -799.0264892578125, 321.9..."


In [45]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[710.2053833007812, -799.0264892578125, 321.9...
1    [[32.499385833740234, -1.5158289670944214, -24...
2    [[124.71330261230469, -38.58845138549805, -42....
3    [[130.8542022705078, -74.68362426757812, 18.32...
4    [[710.2053833007812, -799.0264892578125, 321.9...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3694
Name: count, dtype: int64


In [46]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [47]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[710.2053833007812, -799.0264892578125, 321.9...
1    [[32.499385833740234, -1.5158289670944214, -24...
2    [[124.71330261230469, -38.58845138549805, -42....
3    [[130.8542022705078, -74.68362426757812, 18.32...
4    [[710.2053833007812, -799.0264892578125, 321.9...
Name: embeded_sequence_sum, dtype: object


In [48]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [49]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [50]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [51]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 3690


In [52]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [710.2053833007812, -799.0264892578125, 321.91...
1    [32.499385833740234, -1.5158289670944214, -24....
2    [124.71330261230469, -38.58845138549805, -42.5...
3    [130.8542022705078, -74.68362426757812, 18.322...
4    [710.2053833007812, -799.0264892578125, 321.91...
Name: embeded_sequence_sum, dtype: object


In [53]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3694
Name: count, dtype: int64


In [54]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


        emb_1       emb_2       emb_3        emb_4        emb_5        emb_6  \
0  710.205383 -799.026489  321.919617 -1624.527222  2408.093262 -2533.795166   
1   32.499386   -1.515829  -24.590275   -30.602459    47.787666   -27.378401   
2  124.713303  -38.588451  -42.579781   -39.911690    85.068405   -40.210751   
3  130.854202  -74.683624   18.322701   -92.782379   122.927406   -85.169540   
4  710.205383 -799.026489  321.919617 -1624.527222  2408.093262 -2533.795166   

        emb_7       emb_8       emb_9      emb_10  ...    emb_762     emb_763  \
0 -817.205566 -702.485046 -464.826233 -554.832153  ... -90.683929  149.631775   
1  -21.693262   -6.852709  -29.279053  -34.747868  ... -26.822235    2.004940   
2  -44.931362    6.869379  -48.874641  -76.863480  ...  45.405861   45.218468   
3  -60.637974  -51.506363  -53.193283  -14.298395  ... -42.749687   43.243038   
4 -817.205566 -702.485046 -464.826233 -554.832153  ... -90.683929  149.631775   

       emb_764      emb_765     

In [55]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [56]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[0.7653075456619263, -0.8610199093818665, 0.3...
1    [[1.8055214881896973, -0.08421272039413452, -1...
2    [[3.3706297874450684, -1.042931079864502, -1.1...
3    [[1.9243265390396118, -1.0982885360717773, 0.2...
4    [[0.7653075456619263, -0.8610199093818665, 0.3...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3694
Name: count, dtype: int64


In [57]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [58]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 15697


In [59]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [60]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 3588


In [61]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [0.7653075456619263, -0.8610199093818665, 0.34...
1    [1.8055214881896973, -0.08421272039413452, -1....
2    [3.3706297874450684, -1.042931079864502, -1.15...
3    [1.9243265390396118, -1.0982885360717773, 0.26...
4    [0.7653075456619263, -0.8610199093818665, 0.34...
Name: embeded_sequence_avg, dtype: object


In [62]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3694
Name: count, dtype: int64


In [63]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.765308,-0.861020,0.346896,-1.750568,2.594928,-2.730383,-0.880609,-0.756988,-0.500890,-0.597879,...,-0.097720,0.161241,-2.178341,-3.397756,-2.166048,2.057266,1.184136,1,8077,critical
1,1.805521,-0.084213,-1.366126,-1.700137,2.654870,-1.521022,-1.205181,-0.380706,-1.626614,-1.930437,...,-1.490124,0.111386,-1.366581,-0.949441,-3.066421,1.671481,-1.336844,1,5553,critical
2,3.370630,-1.042931,-1.150805,-1.078694,2.299146,-1.086777,-1.214361,0.185659,-1.320936,-2.077391,...,1.227185,1.222121,-1.098609,-3.634563,-2.632948,0.881600,1.248681,1,9341,critical
3,1.924327,-1.098289,0.269451,-1.364447,1.807756,-1.252493,-0.891735,-0.757447,-0.782254,-0.210271,...,-0.628672,0.635927,-1.499439,-3.881006,-1.574075,2.070461,0.875936,1,10419,critical
4,0.765308,-0.861020,0.346896,-1.750568,2.594928,-2.730383,-0.880609,-0.756988,-0.500890,-0.597879,...,-0.097720,0.161241,-2.178341,-3.397756,-2.166048,2.057266,1.184136,1,12232,critical


In [64]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)